<a href="https://colab.research.google.com/github/Viny2030/HUMAI/blob/main/0_Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/NLP/4_Seq2Seq/Seq2Seq.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>

# Modelos de Secuencia a Secuencia

Los modelos de secuencia a secuencia son modelos de aprendizaje profundo que han logrado mucho éxito en tareas como traducción automática, resumen de texto y subtítulos de imágenes. Google Translate comenzó a usar un modelo de este tipo en producción a finales de 2016.

Un modelo de secuencia a secuencia es un modelo que toma una secuencia de elementos (palabras, letras, características de una imagen, etc.) y genera otra secuencia de elementos.

![Imgur](https://i.imgur.com/nNOUbuN.gif)

# Arquitectura Encoder-Decoder

Para manejar este tipo de entradas y salidas, podemos diseñar una arquitectura con dos componentes principales.

1. El primer componente es un `Encoder` (codificador): toma una secuencia de longitud variable como entrada y la transforma en un estado (también llamado contexto) con una forma fija.
2. El segundo componente es un `Decoder` (encoder): mapea el estado codificado de una forma fija a una secuencia de longitud variable.


![Imgur](https://i.imgur.com/dd2Qril.gif)

En las siguientes celdas vamos a definir una interfaz para el decoder y otra para el decoder. Las interfaces son clases que establecen las responsabilidades básicas de un objeto, pero que dejan que cada objeto decida como implementarlas.

Como vemos en la siguiente celda un Encoder es un modelo que recibe una entrada secuencial X (y algún otro argumento opcional) y genera una salida.

In [ ]:
import torch
import torch.nn as nn
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()

    # Más tarde puede haber argumentos adicionales
    # (por ejemplo, longitud para excluir el relleno)
    def forward(self, X, *args):
        raise NotImplementedError

Por otro lado, el decoder debe tener una función adicional `init_state` para convertir la salida del encoder en el vector estado codificado. Y el forward debe recibir el estado codificado y la entrada X.

In [ ]:
class Decoder(nn.Module):
    """The base decoder interface for the encoder-decoder architecture."""
    def __init__(self):
        super().__init__()

    # Más tarde puede haber argumentos adicionales
    # (por ejemplo, longitud para excluir el relleno)
    def init_state(self, enc_outputs, *args):
        raise NotImplementedError

    def forward(self, X, state):
        raise NotImplementedError

Al final, la arquitectura Encoder-Decoder contiene tanto un encoder como un decoder, con argumentos adicionales opcionales. En la función forward, la salida del encoder se usa para producir el estado codificado, y el decoder usará este estado como una de sus entradas.

In [ ]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, enc_X, dec_X, *args):
        enc_outputs = self.encoder(enc_X, *args)
        dec_state = self.decoder.init_state(enc_outputs, *args)
        # Sólo devuelve la salida del decoder
        return self.decoder(dec_X, dec_state)[0]